In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers, models


In [3]:
df = pd.read_csv('../Data/sample_labels.csv')
df = df[['Image Index', 'Finding Labels']]


In [4]:
label_encoder = LabelEncoder()
df['Labels'] = label_encoder.fit_transform(df['Finding Labels'])

In [5]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
#Image folders and paths
image_folder = '../Data/images2'
train_image_paths = [os.path.join(image_folder, img) for img in train_df['Image Index']]
test_image_paths = [os.path.join(image_folder, img) for img in test_df['Image Index']]


In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20
)

In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
from tensorflow.keras.utils import to_categorical

num_classes = len(label_encoder.classes_)
train_labels_onehot = to_categorical(train_df['Labels'], num_classes=num_classes)
test_labels_onehot = to_categorical(test_df['Labels'], num_classes=num_classes)


In [10]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=image_folder,
    x_col='Image Index',
    y_col='Labels',
    target_size=(224, 224),
    batch_size=15,
    class_mode='raw',
    shuffle=True
)

Found 4484 validated image filenames.


In [11]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=image_folder,
    x_col='Image Index',
    y_col='Labels',
    target_size=(224, 224),
    batch_size=15,
    class_mode='raw',
)

Found 1122 validated image filenames.


In [13]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

In [14]:
from keras import optimizers
optm = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optm, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
history = model.fit(
    train_generator,
    epochs=4,
    validation_data=test_generator
)


Epoch 1/4


2023-12-12 05:54:27.750906: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-12 05:54:28.529726: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f8410013fa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-12 05:54:28.529746: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2023-12-12 05:54:28.534399: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1702340668.607592   10944 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


299/299 [==============================] - 78s 251ms/step - loss: 2.6847 - accuracy: 0.5401 - val_loss: 2.4645 - val_accuracy: 0.5401
Epoch 2/4
299/299 [==============================] - 73s 245ms/step - loss: 2.4764 - accuracy: 0.5437 - val_loss: 2.4610 - val_accuracy: 0.5401
Epoch 3/4
299/299 [==============================] - 74s 247ms/step - loss: 2.4484 - accuracy: 0.5437 - val_loss: 2.4801 - val_accuracy: 0.5401
Epoch 4/4
299/299 [==============================] - 74s 248ms/step - loss: 2.4298 - accuracy: 0.5437 - val_loss: 2.5677 - val_accuracy: 0.5401


In [16]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Loss: {test_loss:.4f}')


75/75 [==============================] - 10s 134ms/step - loss: 2.5677 - accuracy: 0.5401
Test Accuracy: 54.01%
Test Loss: 2.5677


In [17]:
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)
predicted_classes = label_encoder.inverse_transform(predicted_labels)
true_classes = test_df['Labels'].values

for i in range(200):
    print(f"True Class: {true_classes[i]}, Predicted Class: {predicted_classes[i]}")



75/75 [==============================] - 10s 135ms/step
True Class: 234, Predicted Class: No Finding
True Class: 234, Predicted Class: No Finding
True Class: 227, Predicted Class: No Finding
True Class: 234, Predicted Class: No Finding
True Class: 133, Predicted Class: No Finding
True Class: 234, Predicted Class: No Finding
True Class: 234, Predicted Class: No Finding
True Class: 200, Predicted Class: No Finding
True Class: 234, Predicted Class: No Finding
True Class: 234, Predicted Class: No Finding
True Class: 213, Predicted Class: No Finding
True Class: 234, Predicted Class: No Finding
True Class: 234, Predicted Class: No Finding
True Class: 213, Predicted Class: No Finding
True Class: 0, Predicted Class: No Finding
True Class: 162, Predicted Class: No Finding
True Class: 234, Predicted Class: No Finding
True Class: 235, Predicted Class: No Finding
True Class: 234, Predicted Class: No Finding
True Class: 234, Predicted Class: No Finding
True Class: 234, Predicted Class: No Finding
T